In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('autos.csv', encoding = 'ISO-8859-1')

# IRRELEVANTES

df = df.drop('dateCrawled', axis=1)
df = df.drop('dateCreated', axis=1)
df = df.drop('nrOfPictures', axis=1)
df = df.drop('postalCode', axis=1)
df = df.drop('lastSeen', axis=1)


# DESBALANCEADOS 

# df['name'].value_counts()
df = df.drop('name', axis=1)

# df['seller'].value_counts()
df = df.drop('seller', axis=1)

# df['offerType'].value_counts()
df = df.drop('offerType', axis=1)

# INCONSISTENTES

i1 = df.loc[df.price <= 10]
df = df[df.price > 10]

i2 = df.loc[df.price > 350000]
df = df[df.price <= 350000]

# DELETETANDO ALGUNS NULOS

df = df.loc[~pd.isnull(df['vehicleType'])]
df = df.loc[~pd.isnull(df['model'])]


# PREENCHENDO ALGUNS NULO

df.loc[pd.isnull(df['fuelType'])]
df['fuelType'].value_counts

df.loc[pd.isnull(df['gearbox'])]
df['gearbox'].value_counts

df.loc[pd.isnull(df['notRepairedDamage'])]
df['notRepairedDamage'].value_counts

valores = {"fuelType": "benzin", "gearbox": "manuell", "notRepairedDamage": "nein"}
df = df.fillna(valores)

In [ ]:
X = df.iloc[:, 1:].values
y = df['price'].values

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelEncoder = LabelEncoder()

X[:, 0] = labelEncoder.fit_transform(X[:, 0])
X[:, 1] = labelEncoder.fit_transform(X[:, 1])
X[:, 3] = labelEncoder.fit_transform(X[:, 3])
X[:, 5] = labelEncoder.fit_transform(X[:, 5])
X[:, 8] = labelEncoder.fit_transform(X[:, 8])
X[:, 9] = labelEncoder.fit_transform(X[:, 9])
X[:, 10] = labelEncoder.fit_transform(X[:, 10])

ohe = OneHotEncoder(categorical_features=[0, 1, 3, 5, 8, 9, 10])

X = ohe.fit_transform(X).toarray()

In [ ]:
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor

def criar_rede():
    model = Sequential()
    
    model.add(Dense(units=158, activation='relu', input_dim=315))
    model.add(Dense(units=158, activation='relu'))
    model.add(Dense(units=1, activation='linear'))
    
    model.compile(optimizer='adam', loss='mae')
    
    return model

In [ ]:
model = KerasRegressor(build_fn=criar_rede, epochs=1, batch_size = 10000)
result = cross_val_score(estimator=model, X=X, y=y, cv=10, scoring='r2')

print(f'Média: {result.mean()}')
print(f'Média: {result.std()}')